In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proj1_helpers import *
from implementations import *
DATA_FOLDER = '../data/'

In [3]:
from split_data import split_data

In [4]:
y_train, x_train, id_train = load_csv_data(DATA_FOLDER + 'train.csv')
y_test, x_test, id_test = load_csv_data(DATA_FOLDER + 'test.csv')

In [236]:
x_train.filter(lambda v: v==v, x)

AttributeError: 'numpy.ndarray' object has no attribute 'filter'

In [5]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [6]:
np.shape(x_train)

(250000, 30)

In [7]:
id_train

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

### Basic Least Squares

In [361]:
def compare_prediction(w_train, x, y):
    y_pred = predict_labels(w_train, x)
    return (y_pred == y).sum()/len(y_pred)

In [9]:
w_ls, loss_ls = least_squares(y_train, x_train)
np.shape(w_ls)

(30,)

In [10]:
compare_prediction(w_ls, x_train, y_train)

0.74432799999999999

### Stochastic Gradient Descent

In [11]:
np.shape(np.asarray([1]*30)[:,None].reshape(30, 1))

(30, 1)

In [12]:
np.shape(np.asarray([1]*30))

(30,)

In [13]:
initial_w = np.asarray([1]*30)
max_iters = 30
gamma = 0.7

In [354]:
w_sgd, loss_sgd = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)

C:\ML_project\project1\template\implementations.py:10: RuntimeWarning: overflow encountered in square
  from learning_by_penalized_gradient import *


In [355]:
compare_prediction(w_sgd, x_train, y_train)

0.62927200000000005

### Ridge Regression

In [344]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # split data
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    tr_poly = build_poly(x_tr, degree)
    te_poly = build_poly(x_te, degree)
    print(np.shape(x_tr))
    print(np.shape(tr_poly))
    #####
    #tr_poly = np.sum(np.split(tr_poly, degree+1, axis = 1), axis=0)
    #te_poly = np.sum(np.split(te_poly, degree+1, axis = 1), axis=0)
    #####
    
    loss = 1000
    w = 0
    
    for lambda_ in lambdas:
        # ridge regression
        weight, l = ridge_regression(y_tr, tr_poly, lambda_)
        #weight = stack_w(weight, degree)
        mse_test = compute_loss(y_te, te_poly, weight)
        
        if(mse_test < loss):
            loss = mse_test
            w = weight
        
    return w, loss

In [383]:
seed = 55
degree = 6
split_ratio = 0.9

w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_ratio, seed)

(225000, 30)
(225000, 210)


In [372]:
np.shape(y_train)

(250000,)

In [369]:
np.shape(w_r)

(90,)

In [258]:
w_s = np.sum(np.split(w_r, degree + 1),axis=0)
np.shape(w_s)

(30,)

In [359]:
np.shape(build_poly(x_train, degree))

(250000, 120)

In [384]:
compare_prediction(w_r.reshape(len(w_r), 1), build_poly(x_train, degree), y_train.reshape(len(y_train), 1))

0.79979199999999995

### Logistic Regression

In [57]:
def ridge_regression2(y, tx, lamb):
    """implement ridge regression."""
    aI = lamb * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

def ridge_regression3(y, tx, lambda_):
    first_term = tx.T.dot(tx)
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T.dot(y)
    w = np.linalg.solve(left, right)
    loss = compute_loss2(y, tx, w)
    return w, loss;

def compute_loss2(y, tx, w):
    e = y - tx.dot(w)
    return np.mean(e**2)/2.

### Bias-variance

In [234]:
from implementations import least_squares
from split_data import split_data

def bias_variance_demo():
    """The entry."""
    # define parameters
    seeds = range(100)
    num_data = 10000
    ratio_train = 0.005
    degrees = range(1, 10)
    lambdas = np.logspace(-5, 0, 15)
    
    best_seed = 0
    best_degree = 0
    best_lambda = 0
    min_loss = 1000
    for index_seed, seed in enumerate(seeds):
        np.random.seed(seed)
    
        # split data with a specific seed
        x_tr, y_tr, x_te, y_te = split_data(x_train, y_train, ratio_train, seed)
        
        # bias_variance_decomposition
        for index_deg, degree in enumerate(degrees):
            # form data with polynomial degree
            train_poly = build_poly(x_tr, degree)
            test_poly = build_poly(x_te, degree)
            
            w_train, loss_train = least_squares(y_tr, train_poly)

            # calculate the loss for train and test data
            loss_test = compute_loss(y_te, test_poly, w_train)
            
            if(loss_test < min_loss):
                min_loss = loss_test
                best_seed = seed
                #best_degree = degree
                best_lambda = lambda_
    #return best_seed, best_lambda, min_loss
    return best_seed, best_degree ,min_loss        


In [235]:
s, d, l = bias_variance_demo()

LinAlgError: Singular matrix

In [207]:
l

0.34304695760259968

### Cross-validation

In [310]:
from build_k_indices import build_k_indices
from implementations import ridge_regression
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_tr = x[train_indices]
    y_tr = y[train_indices]
    
    x_te = x[test_indices]
    y_te = y[test_indices]

    # form data with polynomial degree
    #train_poly = build_poly(x_tr, degree)
    #test_poly = build_poly(x_te, degree)

    # ridge regression
    
    weights_train, loss_tr = ridge_regression(y_tr, x_tr, lambda_)

    # calculate the loss for train and test data
    loss_te = compute_loss(y_te, x_te, weights_train)

    return loss_tr, loss_te

In [326]:
def cross_validation_demo():
    seed = 55
    degree = 2
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    
    min_mean_loss = 1000
    best_lambdas = np.zeros(k_fold)
    # cross validation
    for i_test in range(k_fold):
        min_loss = 1000
        for lambda_ in lambdas:
            loss_tr, loss_te = cross_validation(y_train, x_train, k_indices, i_test, lambda_, degree)
            
            if(loss_te < min_loss):
                min_loss = loss_te
                best_lambdas[i_test] = lambda_
    lambda_mean = np.mean(best_lambdas)
    return lambda_mean, min_mean_loss

In [327]:
bl, mml = cross_validation_demo()

In [328]:
bl

0.25007499999999999

In [221]:
w, l = ridge_regression(y_train, x_train, bl)

In [222]:
compare_prediction(w, x_train, y_train)

0.74432799999999999